**MODEL - EFFICIENT NET**

**TEAM - CODEBOTS**

In [ ]:
# CHECKING GPU ALLOCATION
|!nvidia-smi

Thu Aug 12 12:12:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# SELECTING GPU FRACTION TO BE UTILIZED WHILE EXECUTING CODE
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
# MOUNTING GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
# IMPORTING ESSENTIAL LIBRARIES
import keras
import tensorflow as tf
from tensorflow import keras
import numpy as np

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from glob import glob



In [ ]:
# IMPORTING EFFICIENT NET MODEL
import efficientnet.tfkeras as efn

In [ ]:
# DRIVE PATH TO TRAIN SET AND TEST SET
train_path = '/content/drive/MyDrive/data healthcare/train'
valid_path = '/content/drive/MyDrive/data healthcare/test'

**BUILDING MODEL**

In [ ]:
# CHOOSING 'IMAGENET' PRE TRAINED WEIGHTS AND REMOVING EXTREME LAYERS

efficientnet = efn.EfficientNetB0(input_shape= (224,224,3), weights = 'imagenet', include_top= False)

# PREVENTING LAYERS NOT GET TRAIN AGAIN
for layer in efficientnet.layers:
  layer.trainable = False



# ADDING FLATTEN LAYER

x = Flatten()(efficientnet.output)

# 5 OUTPUT NODES FOR 5 CLASSES

prediction = Dense(5, activation = 'softmax')(x)

model = Model(inputs = efficientnet.input, outputs = prediction)

model.summary()

# COMPILING MODEL WITH ADAM OPTIMIZER AND CATEGORICAL_CROSSENTROPY LOSS

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = 'accuracy'
)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 4,363,169
Trainable params: 313,605
Non-trainable params: 4,049,564
__________________________________________________________________________________________________


**DATA_PRE_PROCESSING**

In [ ]:
# RESCALING/DATA AUGMENTATION/PREPROCESSING FOR TRAIN,TEST DATA

train_datagen = ImageDataGenerator( rescale= 1./255)

test_datagen = ImageDataGenerator(rescale= 1./255)


training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/data healthcare/train', 
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set =  test_datagen.flow_from_directory('/content/drive/MyDrive/data healthcare/test',
                                             target_size = (224,224),
                                             batch_size =32,
                                             class_mode = 'categorical')

Found 16054 images belonging to 5 classes.
Found 3799 images belonging to 5 classes.


In [ ]:
# FITTING THE MODEL WITH TRAIN, VALIDATION DATA

model.fit_generator(
    training_set, validation_data = test_set,
    epochs =5,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set) 
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
502/502 [==============================] - 245s 488ms/step - loss: 0.1787 - accuracy: 0.9841 - val_loss: 1.6789 - val_accuracy: 0.9216
Epoch 2/5
502/502 [==============================] - 244s 487ms/step - loss: 0.1074 - accuracy: 0.9889 - val_loss: 2.4383 - val_accuracy: 0.9187
Epoch 3/5
502/502 [==============================] - 240s 478ms/step - loss: 0.1045 - accuracy: 0.9900 - val_loss: 2.8785 - val_accuracy: 0.8923
Epoch 4/5
502/502 [==============================] - 244s 487ms/step - loss: 0.1037 - accuracy: 0.9907 - val_loss: 3.8860 - val_accuracy: 0.8776
Epoch 5/5
502/502 [==============================] - 242s 482ms/step - loss: 0.1505 - accuracy: 0.9877 - val_loss: 6.8679 - val_accuracy: 0.8402
